In [2]:
!pip install destine-auth

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 20.8 MB/s eta 0:00:0000:0100:01
  Created wheel for destine-auth: filename=destine_auth-0.1-py3-none-any.whl size=3552 sha256=fa47e4696df24e3e8cbbf97ea1e514bcf8d2f7c4bf53a9d4a579651239573b47
  Stored in directory: /home/jovyan/.cache/pip/wheels/9b/d5/13/781f8378538f7851f6000de2441cfcdb398d2ea83ce7a9610e
Successfully built destine-auth


In [3]:
import json
from io import BytesIO
from urllib.parse import urlencode
import getpass
import pycurl
import requests
from IPython.display import JSON   
import destine_auth as da

In [4]:
DESP_USERNAME = 'username'
DESP_PASSWORD = 'password'
 
token = da.AuthHandler(DESP_USERNAME, DESP_PASSWORD)          
access_token = token.get_token()
 
# Check the status of the request
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token") 

Response code: 200
DEDL/DESP Access Token Obtained Successfully


In [5]:
#token = json.loads(buffer.getvalue().decode())["access_token"]
api_headers = {'Authorization': 'Bearer ' + access_token}
service_root_url = "https://odp.data.destination-earth.eu/odata/v1/"
result = requests.get(service_root_url+"Workflows", headers=api_headers).json()
available_workflows = json.dumps(result,indent=2)
JSON(result)

<IPython.core.display.JSON object>